# Projekt Uczenia Maszynowego - Predykcja Opóźnień Lotów
## Semestr letni 2024/25

### Skład grupy i podział zadań

- **Kamil Arkit, Dawid Chomiak**: Znalezienie i przygotowanie danych
- **Kamil Arkit, Dominik Sobótka**: Trenowanie, testowanie i ocena modelu 1
- **Dawid Chomiak, Łukasz Guziczak**: Trenowanie, testowanie i ocena modelu 2
- **Wszyscy**: Sprawozdanie i wnioski

## 1. Opis danych i motywacja autorów

### Zbiór danych
Wykorzystaliśmy zbiór danych "Flight Delays and Cancellations" z platformy Kaggle (US Department of Transportation), zawierający szczegółowe informacje o lotach w USA za 2015 rok. Dataset składa się z ponad 5.8 miliona rekordów lotów z 31 atrybutami.

**Źródło**: https://www.kaggle.com/datasets/usdot/flight-delays

### Motywacja
Wybór tego problemu był motywowany:
- Praktycznym zastosowaniem w branży lotniczej dla optymalizacji operacji
- Znaczeniem ekonomicznym - opóźnienia kosztują branże miliardy dolarów rocznie
- Korzyściami dla pasażerów - lepsze planowanie podróży
- Bogatym zbiorem danych umożliwiającym zastosowanie różnych technik ML
- Aktualnością problemu - opóźnienia lotów dotykają miliony podróżnych

### Główne pliki danych:
- `flights.csv` - dane o lotach (~5.8M rekordów)
- `airlines.csv` - informacje o liniach lotniczych
- `airports.csv` - dane o lotniskach

### Kluczowe atrybuty (flights.csv):
- `YEAR`, `MONTH`, `DAY` - data lotu
- `DAY_OF_WEEK` - dzień tygodnia
- `AIRLINE` - kod linii lotniczej
- `FLIGHT_NUMBER` - numer lotu
- `ORIGIN_AIRPORT`, `DESTINATION_AIRPORT` - lotniska wylotu i przylotu
- `SCHEDULED_DEPARTURE`, `DEPARTURE_TIME` - planowany i rzeczywisty czas wylotu
- `DEPARTURE_DELAY` - opóźnienie wylotu (w minutach)
- `SCHEDULED_ARRIVAL`, `ARRIVAL_TIME` - planowany i rzeczywisty czas przylotu
- `ARRIVAL_DELAY` - opóźnienie przylotu (w minutach)
- `CANCELLED` - czy lot został odwołany
- `DISTANCE` - dystans lotu
- `AIR_TIME` - czas lotu w powietrzu

### Zmienna docelowa:
`DEPARTURE_DELAY` przekształcone na klasyfikację binarną: opóźnienie >15 minut = 1, inaczej = 0

---

## Wymagania:
```bash
pip install pandas numpy matplotlib seaborn scikit-learn xgboost lightgbm imbalanced-learn kagglehub joblib
```

In [ ]:
import os
import sys

try:
    import kagglehub
except ImportError:
    print("Instaluję kagglehub...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kagglehub"])
    import kagglehub

possible_paths = [
    'data',
    '../data',
    os.path.join(os.getcwd(), 'data'),
]

DATASET_PATH = None

for path in possible_paths:
    if os.path.exists(path) and os.path.exists(os.path.join(path, 'flights.csv')):
        DATASET_PATH = path
        print(f"Znaleziono dane lokalnie w: {DATASET_PATH}")
        break

if DATASET_PATH is None:
    print("Pobieram dane z Kaggle...")
    try:
        DATASET_PATH = kagglehub.dataset_download("usdot/flight-delays")
        print(f"Dane pobrane do: {DATASET_PATH}")
    except Exception as e:
        print(f"Błąd pobierania: {e}")
        raise

required_files = ['flights.csv', 'airlines.csv', 'airports.csv']
missing_files = []
for file in required_files:
    if not os.path.exists(os.path.join(DATASET_PATH, file)):
        missing_files.append(file)

if missing_files:
    print(f"Brakuje plików: {missing_files}")
    raise FileNotFoundError(f"Nie znaleziono wymaganych plików: {missing_files}")
else:
    print("Wszystkie pliki danych są dostępne!")
    print(f"Lokalizacja: {os.path.abspath(DATASET_PATH)}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import (
    accuracy_score, roc_auc_score, f1_score, recall_score, 
    precision_score, confusion_matrix, classification_report, roc_curve
)
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
import warnings
import time
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Biblioteki załadowane pomyślnie!")

## 2. Wczytanie danych i analiza eksploracyjna

In [ ]:
print("Wczytywanie danych...")
try:
    flights = pd.read_csv(os.path.join(DATASET_PATH, 'flights.csv'), nrows=500000)
    airlines = pd.read_csv(os.path.join(DATASET_PATH, 'airlines.csv'))
    airports = pd.read_csv(os.path.join(DATASET_PATH, 'airports.csv'))
    
    print(f"Wczytano {len(flights):,} lotów (sample)")
    print(f"Liczba linii lotniczych: {len(airlines)}")
    print(f"Liczba lotnisk: {len(airports)}")
    
    print("\nPrzykładowe dane:")
    display(flights.head())
    
    print("\nInformacje o danych:")
    print(flights.info())
    
    print("\nBraki danych:")
    missing_data = flights.isnull().sum()
    print(missing_data[missing_data > 0].sort_values(ascending=False))
    
except Exception as e:
    print(f"Błąd wczytywania danych: {e}")
    raise

## 3. Przygotowanie danych podstawowych

In [ ]:
df = flights.copy()

df = df[df['CANCELLED'] == 0]
print(f"Po usunięciu odwołanych: {len(df)} lotów")

key_columns = ['DEPARTURE_DELAY', 'AIRLINE', 'ORIGIN_AIRPORT', 
               'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DISTANCE']
df = df.dropna(subset=key_columns)
print(f"Po usunięciu braków: {len(df)} lotów")

df['DELAYED'] = (df['DEPARTURE_DELAY'] > 15).astype(int)
print(f"\nProcent opóźnionych lotów: {df['DELAYED'].mean()*100:.2f}%")

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
delays_for_plot = df['DEPARTURE_DELAY'][(df['DEPARTURE_DELAY'] >= -30) & (df['DEPARTURE_DELAY'] <= 120)]
plt.hist(delays_for_plot, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=15, color='red', linestyle='--', label='Próg 15 min')
plt.title('Rozkład opóźnień (-30 do 120 min)')
plt.xlabel('Opóźnienie (minuty)')
plt.ylabel('Liczba lotów')
plt.legend()

plt.subplot(1, 3, 2)
extreme_delays = df[df['DEPARTURE_DELAY'] > 300]
plt.hist(extreme_delays['DEPARTURE_DELAY'], bins=30, edgecolor='black', alpha=0.7, color='orange')
plt.title(f'Ekstremalne opóźnienia (>300 min)\nn={len(extreme_delays)}')
plt.xlabel('Opóźnienie (minuty)')
plt.ylabel('Liczba lotów')

plt.subplot(1, 3, 3)
delay_counts = df['DELAYED'].value_counts()
plt.pie(delay_counts.values, labels=['Na czas (≤15 min)', 'Opóźniony (>15 min)'], 
        autopct='%1.1f%%', startangle=90, colors=['lightgreen', 'salmon'])
plt.title('Balans klas')

plt.tight_layout()
plt.show()

print(f"\nMax opóźnienie: {df['DEPARTURE_DELAY'].max():.0f} minut")
print(f"Opóźnienia >300 min: {len(extreme_delays)} ({len(extreme_delays)/len(df)*100:.2f}%)")

## Uzasadnienie decyzji preprocessing

### Usunięte atrybuty i rekordy:
- **Odwołane loty** - skupiamy się na predykcji opóźnień, nie odwołań
- **Ekstremalne opóźnienia (>300 min)** - prawdopodobnie błędy w danych
- **Rekordy z brakami w kluczowych kolumnach**

### Utworzone cechy:
- **DEPARTURE_HOUR** - godzina wylotu ma znaczący wpływ na opóźnienia
- **TIME_OF_DAY** - grupowanie godzin w okresy dnia
- **SEASON** - sezonowość wpływa na ruch lotniczy
- **IS_WEEKEND** - różnice między dniami roboczymi a weekendami
- **DISTANCE_CATEGORY** - kategoryzacja dystansów dla lepszej interpretacji

### Przekształcenia:
- **Klasyfikacja binarna** - opóźnienie >15 minut (standard branżowy)
- **Label encoding** - dla zmiennych kategorycznych z dużą liczbą kategorii
- **Próbkowanie** - dla efektywności obliczeniowej przy zachowaniu reprezentatywności

# ETAP 1: Model Baseline (10% recall)

Prosty model z podstawowymi cechami - punkt startowy dla dalszych ulepszeń.

In [ ]:
print("="*50)
print("ETAP 1: MODEL BASELINE")
print("="*50)

df_stage1 = df.copy()

df_stage1 = df_stage1[(df_stage1['DEPARTURE_DELAY'] >= -30) & 
                      (df_stage1['DEPARTURE_DELAY'] <= 300)]

if len(df_stage1) > 100000:
    df_stage1 = df_stage1.sample(n=100000, random_state=42)

print(f"Używamy {len(df_stage1)} próbek")

df_stage1['DEPARTURE_HOUR'] = df_stage1['SCHEDULED_DEPARTURE'].astype(str).str.zfill(4).str[:2].astype(int)

def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df_stage1['TIME_OF_DAY'] = df_stage1['DEPARTURE_HOUR'].apply(get_time_of_day)

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_stage1['SEASON'] = df_stage1['MONTH'].apply(get_season)
df_stage1['IS_WEEKEND'] = (df_stage1['DAY_OF_WEEK'].isin([6, 7])).astype(int)
df_stage1['DISTANCE_CATEGORY'] = pd.cut(df_stage1['DISTANCE'], 
                                        bins=[0, 500, 1000, 2000, 5000], 
                                        labels=['Short', 'Medium', 'Long', 'Very_Long'])

feature_columns_stage1 = [
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_HOUR',
    'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'DISTANCE', 'IS_WEEKEND', 'TIME_OF_DAY', 'SEASON', 'DISTANCE_CATEGORY'
]

X_stage1 = df_stage1[feature_columns_stage1].copy()
y_stage1 = df_stage1['DELAYED']

categorical_columns = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 
                      'TIME_OF_DAY', 'SEASON', 'DISTANCE_CATEGORY']

for col in categorical_columns:
    le = LabelEncoder()
    X_stage1[col] = le.fit_transform(X_stage1[col].astype(str))

X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X_stage1, y_stage1, test_size=0.2, random_state=42, stratify=y_stage1
)

print(f"\nCechy: {len(feature_columns_stage1)}")
print(f"Zbiór treningowy: {len(X_train1)}, testowy: {len(X_test1)}")
print(f"Procent opóźnień: {y_stage1.mean()*100:.2f}%")

In [ ]:
print("\nTrenowanie modeli baseline...")

rf_baseline = RandomForestClassifier(
    n_estimators=50,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

start = time.time()
rf_baseline.fit(X_train1, y_train1)
print(f"Random Forest - czas trenowania: {time.time()-start:.1f}s")

xgb_baseline = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='logloss'
)

start = time.time()
xgb_baseline.fit(X_train1, y_train1)
print(f"XGBoost - czas trenowania: {time.time()-start:.1f}s")

y_pred_rf1 = rf_baseline.predict(X_test1)
y_pred_xgb1 = xgb_baseline.predict(X_test1)

print("\n=== WYNIKI ETAP 1 (BASELINE) ===")
print("\nRandom Forest:")
print(f"Recall: {recall_score(y_test1, y_pred_rf1)*100:.1f}%")
print(f"Precision: {precision_score(y_test1, y_pred_rf1)*100:.1f}%")
print(f"F1-Score: {f1_score(y_test1, y_pred_rf1):.3f}")

print("\nXGBoost:")
print(f"Recall: {recall_score(y_test1, y_pred_xgb1)*100:.1f}%")
print(f"Precision: {precision_score(y_test1, y_pred_xgb1)*100:.1f}%")
print(f"F1-Score: {f1_score(y_test1, y_pred_xgb1):.3f}")

cm_rf1 = confusion_matrix(y_test1, y_pred_rf1)
cm_xgb1 = confusion_matrix(y_test1, y_pred_xgb1)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

sns.heatmap(cm_rf1, annot=True, fmt='d', cmap='Blues', ax=ax1)
ax1.set_title('Random Forest - Etap 1')
ax1.set_xlabel('Przewidywane')
ax1.set_ylabel('Rzeczywiste')

sns.heatmap(cm_xgb1, annot=True, fmt='d', cmap='Greens', ax=ax2)
ax2.set_title('XGBoost - Etap 1')
ax2.set_xlabel('Przewidywane')
ax2.set_ylabel('Rzeczywiste')

plt.tight_layout()
plt.show()

print("\nPROBLEM: Bardzo niski recall (~10%) - model przewiduje głównie loty na czas!")

# ETAP 2: Data Leakage Model (77.5% recall)

Model z celowym błędem - używa informacji o opóźnieniu (DELAY_LOG) do przewidywania opóźnienia!

In [ ]:
print("="*50)
print("ETAP 2: DATA LEAKAGE MODEL")
print("="*50)

df_stage2 = df.copy()

df_stage2 = df_stage2[(df_stage2['DEPARTURE_DELAY'] >= -30) & 
                      (df_stage2['DEPARTURE_DELAY'] <= 300)]

if len(df_stage2) > 100000:
    df_stage2 = df_stage2.sample(n=100000, random_state=42)

print(f"Używamy {len(df_stage2)} próbek")

df_stage2['DELAYED'] = (df_stage2['DEPARTURE_DELAY'] > 15).astype(int)

df_stage2['DEPARTURE_HOUR'] = df_stage2['SCHEDULED_DEPARTURE'].astype(str).str.zfill(4).str[:2].astype(int)

df_stage2['DELAY_LOG'] = np.log1p(df_stage2['DEPARTURE_DELAY'] + 100)

df_stage2['HOUR_SIN'] = np.sin(2 * np.pi * df_stage2['DEPARTURE_HOUR'] / 24)
df_stage2['HOUR_COS'] = np.cos(2 * np.pi * df_stage2['DEPARTURE_HOUR'] / 24)

df_stage2['IS_RUSH_HOUR'] = (
    ((df_stage2['DEPARTURE_HOUR'] >= 7) & (df_stage2['DEPARTURE_HOUR'] <= 9)) |
    ((df_stage2['DEPARTURE_HOUR'] >= 17) & (df_stage2['DEPARTURE_HOUR'] <= 19))
).astype(int)

df_stage2['IS_WEEKEND'] = (df_stage2['DAY_OF_WEEK'].isin([6, 7])).astype(int)
df_stage2['IS_FRIDAY'] = (df_stage2['DAY_OF_WEEK'] == 5).astype(int)

df_stage2['ORIGIN_CONGESTION'] = df_stage2.groupby('ORIGIN_AIRPORT')['ORIGIN_AIRPORT'].transform('count')
df_stage2['DEST_CONGESTION'] = df_stage2.groupby('DESTINATION_AIRPORT')['DESTINATION_AIRPORT'].transform('count')

airline_delay_rate2 = df_stage2.groupby('AIRLINE')['DELAYED'].mean()
df_stage2['AIRLINE_DELAY_RATE'] = df_stage2['AIRLINE'].map(airline_delay_rate2)

df_stage2['DISTANCE_BIN'] = pd.cut(df_stage2['DISTANCE'], 
                                   bins=[0, 500, 1000, 2000, 5000], 
                                   labels=['Short', 'Medium', 'Long', 'VeryLong'])

feature_columns_stage2 = [
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_HOUR',
    'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'DISTANCE', 'IS_WEEKEND', 'IS_FRIDAY', 'IS_RUSH_HOUR',
    'HOUR_SIN', 'HOUR_COS',
    'ORIGIN_CONGESTION', 'DEST_CONGESTION',
    'AIRLINE_DELAY_RATE', 'DISTANCE_BIN',
    'DELAY_LOG'
]

X_stage2 = df_stage2[feature_columns_stage2].copy()
y_stage2 = df_stage2['DELAYED']

print(f"\nCechy: {len(feature_columns_stage2)} (włącznie z DELAY_LOG - data leakage!)")
print(f"Procent opóźnień: {y_stage2.mean()*100:.2f}%")
print("\nUWAGA: Model używa DELAY_LOG - to jest celowy błąd do demonstracji!")

In [ ]:
FEATURE_LABELS = {
    'MONTH': 'Miesiąc lotu',
    'DAY': 'Dzień miesiąca',
    'DAY_OF_WEEK': 'Dzień tygodnia',
    'DEPARTURE_HOUR': 'Godzina odlotu',
    'DEPARTURE_MINUTE': 'Minuta odlotu',
    
    'AIRLINE': 'Linia lotnicza',
    'ORIGIN_AIRPORT': 'Lotnisko wylotu',
    'DESTINATION_AIRPORT': 'Lotnisko docelowe',
    'DISTANCE': 'Dystans lotu (mile)',
    'LOG_DISTANCE': 'Log(dystans)',
    
    'IS_WEEKEND': 'Czy weekend',
    'IS_FRIDAY': 'Czy piątek',
    'IS_MONDAY': 'Czy poniedziałek',
    'IS_RUSH_HOUR': 'Czy godziny szczytu (7-9, 17-19)',
    'IS_LATE_NIGHT': 'Czy późna noc (22-5)',
    'IS_EARLY_MORNING': 'Czy wczesny ranek (4-6)',
    
    'HOUR_SIN': 'Godzina (składowa sin)',
    'HOUR_COS': 'Godzina (składowa cos)',
    'MONTH_SIN': 'Miesiąc (składowa sin)',
    'MONTH_COS': 'Miesiąc (składowa cos)',
    
    'IS_HOLIDAY_SEASON': 'Czy okres świąteczny',
    'SEASON': 'Sezon roku',
    'TIME_OF_DAY': 'Pora dnia',
    
    'ORIGIN_BUSY': 'Natężenie ruchu - lotnisko wylotu',
    'DEST_BUSY': 'Natężenie ruchu - lotnisko docelowe',
    'ORIGIN_CONGESTION': 'Zagęszczenie - lotnisko wylotu',
    'DEST_CONGESTION': 'Zagęszczenie - lotnisko docelowe',
    'ROUTE': 'Trasa lotu',
    'ROUTE_FREQ': 'Popularność trasy',
    'ROUTE_POPULARITY': 'Częstotliwość trasy',
    
    'AIRLINE_DELAY_RATE': 'Wskaźnik opóźnień linii',
    'ORIGIN_DELAY_RATE': 'Wskaźnik opóźnień lotniska wylotu',
    
    'DISTANCE_BIN': 'Kategoria dystansu',
    'DISTANCE_CATEGORY': 'Kategoria odległości',
    
    'RUSH_AIRLINE': 'Godziny szczytu × wskaźnik linii',
    'HOLIDAY_ORIGIN': 'Święta × wskaźnik lotniska',
    'HOUR_AIRLINE': 'Godzina × wskaźnik linii',
    
    'DELAY_LOG': 'LOG(OPÓŹNIENIE) - DATA LEAKAGE!'
}

def get_feature_label(feature_name):
    """Zwraca opisową etykietę dla cechy"""
    return FEATURE_LABELS.get(feature_name, feature_name)

print("Mapowanie cech utworzone - będzie używane w wykresach")

In [ ]:
categorical_columns = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE_BIN']
for col in categorical_columns:
    le = LabelEncoder()
    X_stage2[col] = le.fit_transform(X_stage2[col].astype(str))

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_stage2, y_stage2, test_size=0.2, random_state=42, stratify=y_stage2
)

smote = SMOTE(random_state=42, sampling_strategy=0.6)
X_train2_smote, y_train2_smote = smote.fit_resample(X_train2, y_train2)

print("\nTrenowanie modelu z data leakage...")
xgb_leakage = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42,
    n_jobs=-1
)

start = time.time()
xgb_leakage.fit(X_train2_smote, y_train2_smote)
print(f"Czas trenowania: {time.time()-start:.1f}s")

y_proba2 = xgb_leakage.predict_proba(X_test2)[:, 1]

thresholds = np.arange(0.3, 0.7, 0.02)
f1_scores = []
for thresh in thresholds:
    y_pred = (y_proba2 >= thresh).astype(int)
    f1_scores.append(f1_score(y_test2, y_pred))

optimal_threshold = thresholds[np.argmax(f1_scores)]
y_pred2 = (y_proba2 >= optimal_threshold).astype(int)

print("\n=== WYNIKI ETAP 2 (DATA LEAKAGE) ===")
print(f"Optymalny threshold: {optimal_threshold:.2f}")
print(f"Recall: {recall_score(y_test2, y_pred2)*100:.1f}%")
print(f"Precision: {precision_score(y_test2, y_pred2)*100:.1f}%")
print(f"F1-Score: {f1_score(y_test2, y_pred2):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test2, y_proba2):.3f}")

cm2 = confusion_matrix(y_test2, y_pred2)
plt.figure(figsize=(8, 6))
sns.heatmap(cm2, annot=True, fmt='d', cmap='Reds')
plt.title('Confusion Matrix - Etap 2 (Data Leakage)')
plt.xlabel('Przewidywane')
plt.ylabel('Rzeczywiste')
plt.show()

importance2 = pd.DataFrame({
    'feature': X_train2.columns,
    'importance': xgb_leakage.feature_importances_
}).sort_values('importance', ascending=False)

importance2['label'] = importance2['feature'].apply(get_feature_label)

plt.figure(figsize=(12, 10))
top_features = importance2.head(15)
plt.barh(range(len(top_features)), top_features['importance'])

plt.yticks(range(len(top_features)), top_features['label'])

plt.xlabel('Ważność cechy', fontsize=12)
plt.title('Top 15 najważniejszych cech - Etap 2 (Data Leakage)', fontsize=14)
plt.gca().invert_yaxis()

for i, (feature, label) in enumerate(zip(top_features['feature'], top_features['label'])):
    if feature == 'DELAY_LOG':
        plt.gca().get_yticklabels()[i].set_color('red')
        plt.gca().get_yticklabels()[i].set_weight('bold')
        plt.gca().get_yticklabels()[i].set_fontsize(12)
    else:
        plt.gca().get_yticklabels()[i].set_fontsize(11)

for i, v in enumerate(top_features['importance']):
    plt.text(v + 0.002, i, f'{v:.3f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\nUWAGA: DELAY_LOG jest najważniejszą cechą - to dowód data leakage!")
print("Model 'oszukuje' używając informacji o opóźnieniu do przewidywania opóźnienia.")

# ETAP 3: Fast Optimized Model (62% recall)

Model po usunięciu data leakage, ale z błędnym usuwaniem outlierów.

In [ ]:
print("="*50)
print("ETAP 3: FAST OPTIMIZED MODEL")
print("="*50)

df_stage3 = df.copy()

df_stage3 = df_stage3[(df_stage3['DEPARTURE_DELAY'] >= -30) & 
                      (df_stage3['DEPARTURE_DELAY'] <= 300)]

print(f"UWAGA: Usunięto {len(df) - len(df_stage3)} lotów z ekstremalnymi opóźnieniami")

if len(df_stage3) > 300000:
    df_stage3 = df_stage3.sample(n=300000, random_state=42)

print(f"Używamy {len(df_stage3)} próbek")

df_stage3['DELAYED'] = (df_stage3['DEPARTURE_DELAY'] > 15).astype(int)

df_stage3['DEPARTURE_HOUR'] = df_stage3['SCHEDULED_DEPARTURE'].astype(str).str.zfill(4).str[:2].astype(int)

df_stage3['HOUR_SIN'] = np.sin(2 * np.pi * df_stage3['DEPARTURE_HOUR'] / 24)
df_stage3['HOUR_COS'] = np.cos(2 * np.pi * df_stage3['DEPARTURE_HOUR'] / 24)

df_stage3['IS_RUSH_HOUR'] = (
    ((df_stage3['DEPARTURE_HOUR'] >= 7) & (df_stage3['DEPARTURE_HOUR'] <= 9)) |
    ((df_stage3['DEPARTURE_HOUR'] >= 17) & (df_stage3['DEPARTURE_HOUR'] <= 19))
).astype(int)

df_stage3['IS_WEEKEND'] = (df_stage3['DAY_OF_WEEK'].isin([6, 7])).astype(int)
df_stage3['IS_FRIDAY'] = (df_stage3['DAY_OF_WEEK'] == 5).astype(int)

df_stage3['ORIGIN_CONGESTION'] = df_stage3.groupby('ORIGIN_AIRPORT')['ORIGIN_AIRPORT'].transform('count')
df_stage3['DEST_CONGESTION'] = df_stage3.groupby('DESTINATION_AIRPORT')['DESTINATION_AIRPORT'].transform('count')

airline_delay_rate3 = df_stage3.groupby('AIRLINE')['DELAYED'].mean()
df_stage3['AIRLINE_DELAY_RATE'] = df_stage3['AIRLINE'].map(airline_delay_rate3)

df_stage3['ROUTE'] = df_stage3['ORIGIN_AIRPORT'] + '_' + df_stage3['DESTINATION_AIRPORT']
df_stage3['ROUTE_POPULARITY'] = df_stage3.groupby('ROUTE')['ROUTE'].transform('count')

df_stage3['DISTANCE_BIN'] = pd.cut(df_stage3['DISTANCE'], 
                                   bins=[0, 500, 1000, 2000, 5000], 
                                   labels=['Short', 'Medium', 'Long', 'VeryLong'])

feature_columns_stage3 = [
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_HOUR',
    'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'DISTANCE', 'IS_WEEKEND', 'IS_FRIDAY', 'IS_RUSH_HOUR',
    'HOUR_SIN', 'HOUR_COS',
    'ORIGIN_CONGESTION', 'DEST_CONGESTION',
    'AIRLINE_DELAY_RATE', 'ROUTE_POPULARITY',
    'DISTANCE_BIN'
]

X_stage3 = df_stage3[feature_columns_stage3].copy()
y_stage3 = df_stage3['DELAYED']

print(f"\nCechy: {len(feature_columns_stage3)} (bez data leakage)")
print(f"Procent opóźnień: {y_stage3.mean()*100:.2f}%")

In [ ]:
categorical_columns = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE_BIN']
for col in categorical_columns:
    le = LabelEncoder()
    X_stage3[col] = le.fit_transform(X_stage3[col].astype(str))

X_train3, X_test3, y_train3, y_test3 = train_test_split(
    X_stage3, y_stage3, test_size=0.2, random_state=42, stratify=y_stage3
)

class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train3), y=y_train3
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

smote = SMOTE(random_state=42, sampling_strategy=0.5)
X_train3_smote, y_train3_smote = smote.fit_resample(X_train3, y_train3)

print("\nTrenowanie modeli...")

rf3 = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=20,
    min_samples_leaf=5,
    class_weight=class_weight_dict,
    random_state=42,
    n_jobs=-1
)
rf3.fit(X_train3_smote, y_train3_smote)

scale_pos_weight = (y_train3 == 0).sum() / (y_train3 == 1).sum()
xgb3 = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)
xgb3.fit(X_train3_smote, y_train3_smote)

lgb3 = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=8,
    learning_rate=0.1,
    class_weight=class_weight_dict,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb3.fit(X_train3_smote, y_train3_smote)

ensemble3 = VotingClassifier(
    estimators=[
        ('rf', rf3),
        ('xgb', xgb3),
        ('lgb', lgb3)
    ],
    voting='soft'
)
ensemble3.fit(X_train3, y_train3)

y_proba3 = ensemble3.predict_proba(X_test3)[:, 1]

thresholds = np.arange(0.3, 0.7, 0.02)
f1_scores = []
for thresh in thresholds:
    y_pred = (y_proba3 >= thresh).astype(int)
    f1_scores.append(f1_score(y_test3, y_pred))

optimal_threshold3 = thresholds[np.argmax(f1_scores)]
y_pred3 = (y_proba3 >= optimal_threshold3).astype(int)

print("\n=== WYNIKI ETAP 3 (FAST OPTIMIZED) ===")
print(f"Optymalny threshold: {optimal_threshold3:.2f}")
print(f"Recall: {recall_score(y_test3, y_pred3)*100:.1f}%")
print(f"Precision: {precision_score(y_test3, y_pred3)*100:.1f}%")
print(f"F1-Score: {f1_score(y_test3, y_pred3):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test3, y_proba3):.3f}")

cm3 = confusion_matrix(y_test3, y_pred3)
plt.figure(figsize=(8, 6))
sns.heatmap(cm3, annot=True, fmt='d', cmap='Oranges')
plt.title('Confusion Matrix - Etap 3 (Fast Optimized)')
plt.xlabel('Przewidywane')
plt.ylabel('Rzeczywiste')
plt.show()

print("\nPROBLEM: Wysoki recall, ale usunęliśmy najtrudniejsze przypadki (>300 min)!")

# ETAP 4: Final Optimized Model (54.4% recall)

Uczciwy model zachowujący WSZYSTKIE opóźnienia, włącznie z ekstremalnymi.

In [ ]:
print("="*50)
print("ETAP 4: FINAL OPTIMIZED MODEL")
print("="*50)

df_stage4 = df.copy()

df_stage4 = df_stage4[df_stage4['DEPARTURE_DELAY'] >= -60]

print(f"Zachowano wszystkie opóźnienia, włącznie z ekstremalnymi")
print(f"Max opóźnienie: {df_stage4['DEPARTURE_DELAY'].max():.0f} minut")
print(f"Opóźnienia >300 min: {(df_stage4['DEPARTURE_DELAY'] > 300).sum()}")

if len(df_stage4) > 300000:
    df_stage4 = df_stage4.sample(n=300000, random_state=42)

df_stage4['DELAYED'] = (df_stage4['DEPARTURE_DELAY'] > 15).astype(int)

df_stage4['DEPARTURE_HOUR'] = df_stage4['SCHEDULED_DEPARTURE'].astype(str).str.zfill(4).str[:2].astype(int)
df_stage4['DEPARTURE_MINUTE'] = df_stage4['SCHEDULED_DEPARTURE'].astype(str).str.zfill(4).str[2:].astype(int)

df_stage4['HOUR_SIN'] = np.sin(2 * np.pi * df_stage4['DEPARTURE_HOUR'] / 24)
df_stage4['HOUR_COS'] = np.cos(2 * np.pi * df_stage4['DEPARTURE_HOUR'] / 24)
df_stage4['MONTH_SIN'] = np.sin(2 * np.pi * df_stage4['MONTH'] / 12)
df_stage4['MONTH_COS'] = np.cos(2 * np.pi * df_stage4['MONTH'] / 12)

df_stage4['IS_RUSH_HOUR'] = (
    ((df_stage4['DEPARTURE_HOUR'] >= 7) & (df_stage4['DEPARTURE_HOUR'] <= 9)) |
    ((df_stage4['DEPARTURE_HOUR'] >= 17) & (df_stage4['DEPARTURE_HOUR'] <= 19))
).astype(int)

df_stage4['IS_LATE_NIGHT'] = (
    (df_stage4['DEPARTURE_HOUR'] >= 22) | (df_stage4['DEPARTURE_HOUR'] <= 5)
).astype(int)

df_stage4['IS_EARLY_MORNING'] = (
    (df_stage4['DEPARTURE_HOUR'] >= 4) & (df_stage4['DEPARTURE_HOUR'] <= 6)
).astype(int)

df_stage4['IS_WEEKEND'] = (df_stage4['DAY_OF_WEEK'].isin([6, 7])).astype(int)
df_stage4['IS_FRIDAY'] = (df_stage4['DAY_OF_WEEK'] == 5).astype(int)
df_stage4['IS_MONDAY'] = (df_stage4['DAY_OF_WEEK'] == 1).astype(int)

df_stage4['IS_HOLIDAY_SEASON'] = (
    ((df_stage4['MONTH'] == 12) & (df_stage4['DAY'] >= 20)) |
    ((df_stage4['MONTH'] == 11) & (df_stage4['DAY'] >= 22) & (df_stage4['DAY'] <= 28)) |
    ((df_stage4['MONTH'] == 7) & (df_stage4['DAY'] <= 7)) |
    ((df_stage4['MONTH'] == 1) & (df_stage4['DAY'] <= 3))
).astype(int)

origin_counts = df_stage4['ORIGIN_AIRPORT'].value_counts()
dest_counts = df_stage4['DESTINATION_AIRPORT'].value_counts()
df_stage4['ORIGIN_BUSY'] = df_stage4['ORIGIN_AIRPORT'].map(origin_counts)
df_stage4['DEST_BUSY'] = df_stage4['DESTINATION_AIRPORT'].map(dest_counts)

df_stage4['ROUTE'] = df_stage4['ORIGIN_AIRPORT'] + '_' + df_stage4['DESTINATION_AIRPORT']
df_stage4['ROUTE_FREQ'] = df_stage4['ROUTE'].map(df_stage4['ROUTE'].value_counts())

airline_delay_rate = df_stage4.groupby('AIRLINE')['DELAYED'].mean()
df_stage4['AIRLINE_DELAY_RATE'] = df_stage4['AIRLINE'].map(airline_delay_rate)

origin_delay_rate = df_stage4.groupby('ORIGIN_AIRPORT')['DELAYED'].mean()
df_stage4['ORIGIN_DELAY_RATE'] = df_stage4['ORIGIN_AIRPORT'].map(origin_delay_rate)

df_stage4['DISTANCE_BIN'] = pd.cut(df_stage4['DISTANCE'], 
                                   bins=[0, 500, 1000, 2000, 5000], 
                                   labels=['Short', 'Medium', 'Long', 'VeryLong'])

df_stage4['RUSH_AIRLINE'] = df_stage4['IS_RUSH_HOUR'] * df_stage4['AIRLINE_DELAY_RATE']
df_stage4['HOLIDAY_ORIGIN'] = df_stage4['IS_HOLIDAY_SEASON'] * df_stage4['ORIGIN_DELAY_RATE']
df_stage4['HOUR_AIRLINE'] = df_stage4['DEPARTURE_HOUR'] * df_stage4['AIRLINE_DELAY_RATE'] / 24

feature_columns_stage4 = [
    'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_HOUR',
    'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE',
    
    'IS_WEEKEND', 'IS_FRIDAY', 'IS_MONDAY', 'IS_RUSH_HOUR', 
    'IS_LATE_NIGHT', 'IS_EARLY_MORNING',
    'HOUR_SIN', 'HOUR_COS', 'MONTH_SIN', 'MONTH_COS',
    
    'IS_HOLIDAY_SEASON',
    
    'ORIGIN_BUSY', 'DEST_BUSY', 'ROUTE_FREQ',
    'AIRLINE_DELAY_RATE', 'ORIGIN_DELAY_RATE',
    
    'DISTANCE_BIN',
    
    'RUSH_AIRLINE', 'HOLIDAY_ORIGIN', 'HOUR_AIRLINE'
]

X_stage4 = df_stage4[feature_columns_stage4].copy()
y_stage4 = df_stage4['DELAYED']

print(f"\nCechy: {len(feature_columns_stage4)}")
print(f"Procent opóźnień: {y_stage4.mean()*100:.2f}%")

In [ ]:
categorical_columns = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE_BIN']
for col in categorical_columns:
    le = LabelEncoder()
    X_stage4[col] = le.fit_transform(X_stage4[col].astype(str))

X_train4, X_test4, y_train4, y_test4 = train_test_split(
    X_stage4, y_stage4, test_size=0.2, random_state=42, stratify=y_stage4
)

class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train4), y=y_train4
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

smote = SMOTE(random_state=42, sampling_strategy=0.6)
X_train4_smote, y_train4_smote = smote.fit_resample(X_train4, y_train4)

print("\nTrenowanie finalnego modelu XGBoost...")
scale_pos_weight = (y_train4 == 0).sum() / (y_train4 == 1).sum()

xgb_final = xgb.XGBClassifier(
    n_estimators=150,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    gamma=0.1,
    random_state=42,
    n_jobs=-1
)

start = time.time()
xgb_final.fit(X_train4_smote, y_train4_smote)
print(f"Czas trenowania: {time.time()-start:.1f}s")

y_proba4 = xgb_final.predict_proba(X_test4)[:, 1]

thresholds = np.arange(0.3, 0.7, 0.02)
f1_scores = []
for thresh in thresholds:
    y_pred = (y_proba4 >= thresh).astype(int)
    f1_scores.append(f1_score(y_test4, y_pred))

optimal_threshold4 = thresholds[np.argmax(f1_scores)]
y_pred4 = (y_proba4 >= optimal_threshold4).astype(int)

print("\n=== WYNIKI ETAP 4 (FINAL MODEL) ===")
print(f"Optymalny threshold: {optimal_threshold4:.2f}")
print(f"Recall: {recall_score(y_test4, y_pred4)*100:.1f}%")
print(f"Precision: {precision_score(y_test4, y_pred4)*100:.1f}%")
print(f"F1-Score: {f1_score(y_test4, y_pred4):.3f}")
print(f"ROC-AUC: {roc_auc_score(y_test4, y_proba4):.3f}")

test_indices = X_test4.index
extreme_delays_mask = df_stage4.loc[test_indices, 'DEPARTURE_DELAY'] > 300
if extreme_delays_mask.sum() > 0:
    extreme_y_true = y_test4[extreme_delays_mask]
    extreme_y_pred = y_pred4[extreme_delays_mask]
    extreme_recall = recall_score(extreme_y_true, extreme_y_pred)
    print(f"\nRecall dla ekstremalnych opóźnień (>300 min): {extreme_recall*100:.1f}%")
    print(f"Wykryto {extreme_y_pred.sum()}/{len(extreme_y_true)} ekstremalnych opóźnień")

cm4 = confusion_matrix(y_test4, y_pred4)
plt.figure(figsize=(8, 6))
sns.heatmap(cm4, annot=True, fmt='d', cmap='Greens')
plt.title('Confusion Matrix - Etap 4 (Final Model)')
plt.xlabel('Przewidywane')
plt.ylabel('Rzeczywiste')
plt.show()

importance4 = pd.DataFrame({
    'feature': X_train4.columns,
    'importance': xgb_final.feature_importances_
}).sort_values('importance', ascending=False)

importance4['label'] = importance4['feature'].apply(get_feature_label)

plt.figure(figsize=(12, 10))
top_features = importance4.head(15)
plt.barh(range(len(top_features)), top_features['importance'])

plt.yticks(range(len(top_features)), top_features['label'])

plt.xlabel('Ważność cechy', fontsize=12)
plt.title('Top 15 najważniejszych cech - Final Model (Uczciwy model)', fontsize=14)
plt.gca().invert_yaxis()

colors = []
for feature in top_features['feature']:
    if feature in ['IS_RUSH_HOUR', 'IS_WEEKEND', 'IS_FRIDAY', 'IS_MONDAY', 
                   'IS_LATE_NIGHT', 'IS_EARLY_MORNING', 'IS_HOLIDAY_SEASON']:
        colors.append('coral')
    elif feature in ['HOUR_SIN', 'HOUR_COS', 'MONTH_SIN', 'MONTH_COS']:
        colors.append('lightsalmon')
    elif feature in ['MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_HOUR']:
        colors.append('peachpuff')
    elif 'ORIGIN' in feature or 'DEST' in feature or 'AIRPORT' in feature:
        colors.append('skyblue')
    elif 'AIRLINE' in feature:
        colors.append('lightgreen')
    elif 'DISTANCE' in feature:
        colors.append('gold')
    elif 'ROUTE' in feature:
        colors.append('plum')
    elif feature in ['RUSH_AIRLINE', 'HOLIDAY_ORIGIN', 'HOUR_AIRLINE']:
        colors.append('lightcoral')
    else:
        colors.append('lightgray')

bars = plt.barh(range(len(top_features)), top_features['importance'], color=colors)

for i, v in enumerate(top_features['importance']):
    plt.text(v + 0.002, i, f'{v:.3f}', va='center', fontsize=10)

from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='coral', label='Cechy czasowe (binarne)'),
    Patch(facecolor='lightsalmon', label='Cechy czasowe (cykliczne)'),
    Patch(facecolor='peachpuff', label='Cechy czasowe (podstawowe)'),
    Patch(facecolor='skyblue', label='Cechy lotniskowe'),
    Patch(facecolor='lightgreen', label='Cechy linii lotniczych'),
    Patch(facecolor='gold', label='Cechy dystansu'),
    Patch(facecolor='plum', label='Cechy tras'),
    Patch(facecolor='lightcoral', label='Cechy interakcyjne/ryzyko')
]
plt.legend(handles=legend_elements, loc='lower right', fontsize=9, ncol=2)

plt.tight_layout()
plt.show()

print("\nModel uczciwie radzi sobie ze WSZYSTKIMI opóźnieniami")
print("Najważniejsze cechy są związane z czasem (godziny szczytu) i lotniskami")
print("\nOpisy TOP 5 najważniejszych cech:")
for i, row in top_features.head(5).iterrows():
    print(f"{i+1}. {row['feature']}: {row['label']} (ważność: {row['importance']:.3f})")

## Podsumowanie: Porównanie wszystkich etapów

In [ ]:
results_summary = pd.DataFrame({
    'Etap': ['1: Baseline', '2: Data Leakage', '3: Fast Optimized', '4: Final Model'],
    'Recall': [
        recall_score(y_test1, y_pred_xgb1)*100,
        recall_score(y_test2, y_pred2)*100,
        recall_score(y_test3, y_pred3)*100,
        recall_score(y_test4, y_pred4)*100
    ],
    'F1-Score': [
        f1_score(y_test1, y_pred_xgb1),
        f1_score(y_test2, y_pred2),
        f1_score(y_test3, y_pred3),
        f1_score(y_test4, y_pred4)
    ],
    'Cechy': [12, 27, 21, 28],
    'Problem': [
        'Zbyt prosty model',
        'Data leakage (DELAY_LOG)',
        'Usunięto outliery >300 min',
        'Uczciwy model ze wszystkim'
    ]
})

print("=== PODSUMOWANIE WSZYSTKICH ETAPÓW ===")
print(results_summary.to_string(index=False))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

bars1 = ax1.bar(results_summary['Etap'], results_summary['Recall'], 
                color=['blue', 'red', 'orange', 'green'])
ax1.set_ylabel('Recall (%)')
ax1.set_title('Ewolucja Recall przez etapy')
ax1.set_ylim(0, 110)

for i, bar in enumerate(bars1):
    height = bar.get_height()
    if height > 90:
        ax1.text(bar.get_x() + bar.get_width()/2., height - 5,
                 f'{height:.1f}%', ha='center', va='top', 
                 color='white', fontweight='bold')
    else:
        ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                 f'{height:.1f}%', ha='center', va='bottom')

bars2 = ax2.bar(results_summary['Etap'], results_summary['F1-Score'], 
                color=['blue', 'red', 'orange', 'green'])
ax2.set_ylabel('F1-Score')
ax2.set_title('Ewolucja F1-Score przez etapy')
ax2.set_ylim(0, 1.0)

for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height - 0.03,
             f'{height:.3f}', ha='center', va='top',
             color='white', fontweight='bold', fontsize=11)

ax1.annotate('Podejrzane!', 
            xy=(1, results_summary.loc[1, 'Recall']), 
            xytext=(1, 85),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            ha='center', fontsize=10, color='red', fontweight='bold')

ax2.annotate('Sztucznie wysoki\n(data leakage)', 
            xy=(1, results_summary.loc[1, 'F1-Score']), 
            xytext=(1, 0.85),
            arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
            ha='center', fontsize=9, color='red')

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))

fpr1, tpr1, _ = roc_curve(y_test1, xgb_baseline.predict_proba(X_test1)[:, 1])
fpr2, tpr2, _ = roc_curve(y_test2, y_proba2)
fpr3, tpr3, _ = roc_curve(y_test3, y_proba3)
fpr4, tpr4, _ = roc_curve(y_test4, y_proba4)

plt.plot(fpr1, tpr1, label=f'Etap 1: Baseline (AUC = {roc_auc_score(y_test1, xgb_baseline.predict_proba(X_test1)[:, 1]):.3f})', linewidth=2)
plt.plot(fpr2, tpr2, label=f'Etap 2: Data Leakage (AUC = {roc_auc_score(y_test2, y_proba2):.3f})', linewidth=2, linestyle='--')
plt.plot(fpr3, tpr3, label=f'Etap 3: Fast Optimized (AUC = {roc_auc_score(y_test3, y_proba3):.3f})', linewidth=2)
plt.plot(fpr4, tpr4, label=f'Etap 4: Final Model (AUC = {roc_auc_score(y_test4, y_proba4):.3f})', linewidth=3)

plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Losowy klasyfikator')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Krzywe ROC - Porównanie wszystkich etapów')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

print("\n=== KLUCZOWE WNIOSKI ===")
print("1. Etap 1 (Baseline): Zbyt konserwatywny model - tylko 10% recall")
print("2. Etap 2 (Data Leakage): Fałszywie wysoki recall 77.5% przez użycie DELAY_LOG")
print("3. Etap 3 (Fast Optimized): Dobry recall 62%, ale osiągnięty przez usunięcie trudnych przypadków")
print("4. Etap 4 (Final Model): Uczciwy recall 54.4% na WSZYSTKICH danych")
print("\nNajlepszy uczciwy model: XGBoost z 28 cechami, F1=0.491, ROC-AUC=0.769")

## Analiza błędów i dalsze kroki

Zobaczmy, gdzie model finalny ma największe problemy.

In [ ]:
test_df = df_stage4.loc[X_test4.index].copy()
test_df['y_true'] = y_test4
test_df['y_pred'] = y_pred4
test_df['y_proba'] = y_proba4

false_negatives = test_df[(test_df['y_true'] == 1) & (test_df['y_pred'] == 0)]
print(f"False Negatives (niewykryte opóźnienia): {len(false_negatives)}")

delay_bins = [15, 30, 60, 120, 300, 2000]
delay_labels = ['15-30 min', '30-60 min', '60-120 min', '120-300 min', '>300 min']

test_df['DELAY_BIN'] = pd.cut(test_df['DEPARTURE_DELAY'], bins=delay_bins, labels=delay_labels, include_lowest=False)

recall_by_delay = test_df[test_df['y_true'] == 1].groupby('DELAY_BIN').apply(
    lambda x: (x['y_pred'] == 1).sum() / len(x) * 100
)

plt.figure(figsize=(10, 6))
recall_by_delay.plot(kind='bar', color='coral')
plt.title('Recall według wielkości opóźnienia')
plt.xlabel('Kategoria opóźnienia')
plt.ylabel('Recall (%)')
plt.xticks(rotation=45)
plt.axhline(y=50, color='red', linestyle='--', alpha=0.5)

for i, v in enumerate(recall_by_delay):
    plt.text(i, v + 1, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.show()

print("\nRecall według wielkości opóźnienia:")
for delay_cat, recall in recall_by_delay.items():
    print(f"{delay_cat}: {recall:.1f}%")

print("\n=== LOTNISKA Z NAJNIŻSZYM RECALL ===")
airport_performance = test_df[test_df['y_true'] == 1].groupby('ORIGIN_AIRPORT').agg({
    'y_pred': ['sum', 'count']
})
airport_performance.columns = ['detected', 'total']
airport_performance['recall'] = airport_performance['detected'] / airport_performance['total'] * 100
airport_performance = airport_performance[airport_performance['total'] >= 10]

worst_airports = airport_performance.nsmallest(10, 'recall')
print(worst_airports[['total', 'detected', 'recall']].round(1))

print("\n=== PROPOZYCJE DALSZYCH ULEPSZEŃ ===")
print("1. Model dwuetapowy:")
print("   - Etap 1: Klasyfikacja normal/extreme delay")
print("   - Etap 2: Dedykowane modele dla każdej grupy")
print("\n2. Dodatkowe cechy:")
print("   - Dane pogodowe (można symulować na podstawie sezonu/lokalizacji)")
print("   - Agregacje historyczne (średnie opóźnienie na trasie ostatnie 7 dni)")
print("   - Cechy ekonomiczne (ceny paliwa, wskaźniki)")
print("\n3. Techniki modelowania:")
print("   - Stacking ensemble z meta-learnerem")
print("   - Custom loss function z większą wagą dla dużych opóźnień")
print("   - Neural network jako dodatkowy model")

## Wnioski i podsumowanie końcowe

### Wyniki modelowania
1. **Etap 1 - Baseline**: Prosty model z podstawowymi cechami osiągnął tylko 10% recall
2. **Etap 2 - Data Leakage**: Celowy błąd (DELAY_LOG) pokazał niebezpieczeństwo wycieku danych
3. **Etap 3 - Fast Optimized**: 62% recall przez usunięcie trudnych przypadków
4. **Etap 4 - Final Model**: Uczciwy 54.4% recall na wszystkich danych

### Kluczowe obserwacje
- **Godzina wylotu** (IS_RUSH_HOUR) jest najważniejszą cechą
- **Cechy czasowe** dominują w top 10 (HOUR_SIN, HOUR_COS, IS_WEEKEND)
- Model ma problemy z ekstremalnymi opóźnieniami (>300 min)
- XGBoost okazał się najlepszym algorytmem

### Rekomendacje
1. **Dla linii lotniczych**:
   - Zwrócić uwagę na zarządzanie harmonogramem w godzinach szczytu (7-9, 17-19)
   - Analiza konkretnych lotnisk z największymi opóźnieniami
   - Sezonowe dostosowanie liczby lotów

2. **Dla pasażerów**:
   - Preferować loty poranne dla minimalizacji ryzyka opóźnień
   - Unikać lotów w piątki i w okresie letnim
   - Brać pod uwagę historię opóźnień konkretnych tras

### Dalsze kroki
- Model dwuetapowy dla ekstremalnych opóźnień
- Włączenie danych pogodowych
- Stacking ensemble z meta-learnerem
- Implementacja w systemie czasu rzeczywistego

In [ ]:
import joblib

joblib.dump(xgb_final, 'best_flight_delay_model.pkl')

model_metadata = {
    'model_type': 'XGBoost',
    'features': feature_columns_stage4,
    'n_features': len(feature_columns_stage4),
    'optimal_threshold': float(optimal_threshold4),
    'performance': {
        'recall': float(recall_score(y_test4, y_pred4)),
        'precision': float(precision_score(y_test4, y_pred4)),
        'f1_score': float(f1_score(y_test4, y_pred4)),
        'roc_auc': float(roc_auc_score(y_test4, y_proba4))
    },
    'training_samples': len(X_train4),
    'test_samples': len(X_test4)
}

import json
with open('model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)

print("Model zapisany jako 'best_flight_delay_model.pkl'")
print("Metadane zapisane jako 'model_metadata.json'")
print("\n=== PROJEKT ZAKOŃCZONY ===")
print(f"Najlepszy model: {model_metadata['model_type']}")
print(f"F1-Score: {model_metadata['performance']['f1_score']:.3f}")
print(f"ROC-AUC: {model_metadata['performance']['roc_auc']:.3f}")
print(f"Recall: {model_metadata['performance']['recall']*100:.1f}%")